In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\shantanu\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import unittest, time, random
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import numpy as np
from tqdm import tqdm


In [3]:
# path to chromedriver
chromedriver_path = r"‪D:\Shantanu Sem 7\Capstone Project\Capstone Sample\flight price prediction\chromedriver_win32\chromedriver.exe"

In [4]:
# launching the driver
driver = webdriver.Chrome(chromedriver_path)

C:\Users\shantanu\AppData\Local\Temp/ipykernel_8456/1365387960.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver_path)


In [5]:
# get user input for routes
sources = []
destinations = []
print("Please enter -1 when done.")
print("-"*10)
while True:
    sources.append(input("From which city?\n"))
    if "-1" in sources: 
        sources.pop(-1)
        break
    destinations.append(input("Where to?\n"))
    if "-1" in destinations: 
        sources.pop(-1)
        destinations.pop(-1)
        break
    print("-"*10)

print("\nRoutes:")
for i in range(len(sources)):
    print(f"{sources[i]} => {destinations[i]}")

Please enter -1 when done.
----------
From which city?
BOM
Where to?
MAA
----------
From which city?
MAA
Where to?
BOM
----------
From which city?
BOM
Where to?
CCU
----------
From which city?
CCU
Where to?
BOM
----------
From which city?
BOM
Where to?
DEL
----------
From which city?
DEL
Where to?
BOM
----------
From which city?
CCU
Where to?
DEL
----------
From which city?
DEL
Where to?
CCU
----------
From which city?
CCU
Where to?
MAA
----------
From which city?
MAA
Where to?
CCU
----------
From which city?
DEL
Where to?
MAA
----------
From which city?
MAA
Where to?
DEL
----------
From which city?
-1

Routes:
BOM => MAA
MAA => BOM
BOM => CCU
CCU => BOM
BOM => DEL
DEL => BOM
CCU => DEL
DEL => CCU
CCU => MAA
MAA => CCU
DEL => MAA
MAA => DEL


In [6]:
# get user input for period (start and end date)
start_date = np.datetime64(input('Start Date, Please use YYYY-MM-DD format only '))
end_date = np.datetime64(input('End Date, Please use YYYY-MM-DD format only '))
days = end_date - start_date
num_days = days.item().days

Start Date, Please use YYYY-MM-DD format only 2022-12-07
End Date, Please use YYYY-MM-DD format only 2022-12-14


In [7]:
## Airlines
def get_airlines(soup):
    airline = []
    airlines = soup.find_all('span',class_='codeshares-airline-names',text=True)
    for i in airlines:
        airline.append(i.text)
    return airline


#Total Stops
def get_total_stops(soup):
    stops_list = []
    stops = soup.find_all('div',class_='section stops')

    for i in stops:
        for j in i.find_all('span',class_='stops-text'):
               stops_list.append(j.text)
    return stops_list

## Price
def get_price(soup):
    prices = []
    price = soup.find_all('div',class_='Flights-Results-FlightPriceSection right-alignment sleek')

    for i in price:
        for j in i.find_all('span', class_='price-text'):
            prices.append(j.text)
    return prices


## Duration
def get_duration(soup):
    duration_list = []
    duration = soup.find_all('div' , class_='section duration allow-multi-modal-icons')
    for i in duration:
        for j in i.find_all('div',class_='top'):
            duration_list.append(j.text)
    return duration_list

In [8]:
for i in range(len(sources)):
    column_names = ["Airline", "Source", "Destination","Duration" ,"Total stops", "Price","Date"]
    df = pd.DataFrame(columns = column_names)
    for j in tqdm(range(num_days+1)):
        
        # close and open driver every 10 days to avoid captcha
        if j % 10 == 0:
            driver.quit()
            driver = webdriver.Chrome(executable_path= chromedriver_path)
            #driver = webdriver.Chrome(chromedriver_path)#, chrome_options=chromeOptions)
            
        url = f"https://www.en.kayak.sa/flights/{sources[i]}-{destinations[i]}/{start_date+j}"
        driver.get(url)
        sleep(15)
        
        # click show more button to get all flights
        try:
            show_more_button = driver.find_element_by_xpath('//a[@class = "moreButton"]')
        except:
            
            # in case a captcha appears, require input from user so that the for loop pauses and the user can continue the
            # loop after solving the captcha
            input("Please solve the captcha then enter anything here to resume scraping.")
            
        while True:
            try:
                show_more_button.click()
                driver.implicitly_wait(10)
            except:
                break
    
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        airlines = get_airlines(soup)
        total_stops = get_total_stops(soup)
        prices = get_price(soup)
        duration = get_duration(soup)
        df = df.append(pd.DataFrame({
            'Airline': airlines,
            'Duration': duration,
            'Total stops' : total_stops,
            'Price' : prices,
            'Date' : start_date+j
                                    }))
        
    df['Source'] = sources[i]
    df['Destination'] = destinations[i]
    df = df.replace('\n','', regex=True)
    df = df.reset_index(drop = True)
    
    # save data as csv file for each route
    df.to_csv(f'{sources[i]}_{destinations[i]}.csv',index=False)
    print(f"Succesfully saved {sources[i]} => {destinations[i]} route as {sources[i]}_{destinations[i]}.csv ")
    
driver.quit()

  0%|                                                                                            | 0/8 [00:00<?, ?it/s]C:\Users\shantanu\AppData\Local\Temp/ipykernel_8456/2740019645.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path= chromedriver_path)


Please solve the captcha then enter anything here to resume scraping.


 12%|██████████▌                                                                         | 1/8 [00:36<04:12, 36.12s/it]

Please solve the captcha then enter anything here to resume scraping.


 25%|█████████████████████                                                               | 2/8 [00:53<02:32, 25.35s/it]

...

Please solve the captcha then enter anything here to resume scraping.


 88%|█████████████████████████████████████████████████████████████████████████▌          | 7/8 [02:24<00:19, 19.44s/it]

Please solve the captcha then enter anything here to resume scraping.


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [02:51<00:00, 21.38s/it]


Succesfully saved MAA => DEL route as MAA_DEL.csv 
